In [19]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.webmd.com/drugs/drugreview-94831-revlimid-oral?drugid=94831'
headers = {'User-Agent': 'Chrome/50.0.2661.102 Safari/537.36'}
page_no = 1
response = requests.get(url, headers=headers)
li1,li2,li4,sat,eou,eff,names,ages,genders,med_period,status=[],[],[],[],[],[],[],[],[],[],[]
if response.status_code == 200:
    for i in range(4):
        new_url = url + "&page=" + str(i+1)
        response = requests.get(new_url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        details = soup.find_all("div", {"class": "details"})
        overallratings = soup.find_all("div", {"class":"overall-rating"})
        ratings = soup.find_all("div", {"class":"categories"})
        reviews = soup.find_all("div", {"class":"description"})
        for div in soup.find_all('div',class_='details'):
            values = div.find_all('span')
            if len(values) == 3:
                name = values[0].text.replace('|','') 
                age = ''
                gender = ''
                med_p = div.contents[3][18:].strip().replace('|','') 
                reviewer = values[2].text 
            else:
                name = values[0].text.replace('|','')  if len (values) >= 1 else ''
                age = values[1].text.replace('|','')  if len(values) >= 2 else ''
                gender = values[2].text.replace('|','')  if len(values) >= 3 else ''
                med_p = div.contents[3][18:].strip().replace('|','') 
                reviewer = values[3].text if len(values) >=4 else ''
            names.append(name)
            ages.append(age)
            genders.append(gender)
            med_period.append(med_p)
            status.append(reviewer)
        for rt in overallratings:
            li2.append(rt.text[-4:])
        for rating in ratings:
            aria_valuenow_values = [div.get('aria-valuenow') for div in rating.findAll('div', {'aria-valuenow': True})]
            eff.append(int(aria_valuenow_values[0]))
            eou.append(int(aria_valuenow_values[1]))
            sat.append(int(aria_valuenow_values[2]))
        for container in soup.find_all("div", {"class": "review-details"}):
            try:
                value = container.find("div", {"class": "description"}).text.strip()
            except AttributeError:
                value = "N/A"
            li4.append(value)
else:
    print(f'Request failed with status code {response.status_code}')


In [21]:
import csv
data = list(zip(names,ages,genders,med_period,status, li2, eff,eou,sat,li4))
with open('revlimid_drugreview.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Name','Age','Gender','Medication Period','Status', 'Overall Rating', 'Effectiveness','Ease of Use','Satisfaction','Reviews'])

    for row in data:
        writer.writerow(row)
